## Predicting The Next Book-to-Movie Adaptation 
What book characteristics are correlated to successful movie adaptations? 

In [21]:
# Import dependencies
import pandas as pd
import matplotlib as pyplot
import numpy as np

In [22]:
# Reference csvs file path
goodreads_path = "Raw_Data/books.csv"
keywords_path = "Raw_Data/keywords.csv"
movies_path = "Raw_Data/movies_metadata.csv"
ratings_path = "Raw_Data/ratings.csv"

# Import csv files as DataFrames
goodreads_df = pd.read_csv(goodreads_path, encoding="utf-8")
keywords_df = pd.read_csv(keywords_path, encoding="utf-8")
movies_df = pd.read_csv(movies_path, encoding="utf-8")
ratings_df = pd.read_csv(ratings_path, encoding="utf-8")

C:\Users\laure\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
adaptations = keywords_df[['based on novel' in row for row in keywords_df['keywords']]]
adaptations.head()

,id,keywords
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
16,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '..."
20,8012,"[{'id': 395, 'name': 'gambling'}, {'id': 416, ..."
70,11859,"[{'id': 258, 'name': 'bomb'}, {'id': 416, 'nam..."
136,39428,"[{'id': 531, 'name': 'southern usa'}, {'id': 8..."


In [25]:
adaptations['id'] = adaptations['id'].astype('str')
adaptations['id'].dtype;

<ipython-input-25-692f3f35c53a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adaptations['id'] = adaptations['id'].astype('str')


In [26]:
# Inner join adaptations on id to movies_df (if not unique id, find correct id), 
adaptations_merge = pd.merge(adaptations, movies_df,  how = 'inner', on = 'id')
adaptations_merge = adaptations_merge.set_index('id')
adaptations_merge.head(2)
# ratings_df (may need to switch id), goodreads_df (based on title, may need try except)

,keywords,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,
31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,NaN,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,tt0114388,en,Sense and Sensibility,"Rich Mr. Dashwood dies, leaving his second wif...",...,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,7.2,364.0


In [27]:
# drop belongs_to_collection, homepage, tagline, 
# imdb_id, overview, poster_path, video, vote_count, vote_average, popularity
# filter by released
adaptations_merge = adaptations_merge[['keywords', 'adult', 'budget', 'genres', 
                                       'original_language', 'original_title', 'release_date', 
                                      'revenue', 'runtime', 'spoken_languages', 'status', 'title']]
adaptations_merge = adaptations_merge[adaptations_merge['status'] == 'Released']
adaptations_merge = adaptations_merge.dropna(how = 'any')
# 829 rows
adaptations_merge.head()

,keywords,adult,budget,genres,original_language,original_title,release_date,revenue,runtime,spoken_languages,status,title
id,,,,,,,,,,,,
31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale
4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,Sense and Sensibility,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Sense and Sensibility
8012,"[{'id': 395, 'name': 'gambling'}, {'id': 416, ...",False,30250000,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",en,Get Shorty,1995-10-20,115101622.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Get Shorty
11859,"[{'id': 258, 'name': 'bomb'}, {'id': 416, 'nam...",False,50000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,Fair Game,1995-11-03,11534477.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Fair Game
39428,"[{'id': 531, 'name': 'southern usa'}, {'id': 8...",False,0,"[{'id': 18, 'name': 'Drama'}]",en,The Neon Bible,1995-08-23,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Neon Bible


In [28]:
# Rename columns in ratings_df to merge on 'id'
ratings_df = ratings_df.rename(columns = {'userId': "userId", 'movieId': 'id', 
                                'rating': 'rating', 'timestamp':'timestamp'})
# Drop columns
ratings_df = ratings_df[['id', 'rating']]

# Convert int to str
ratings_df['id'] = ratings_df['id'].astype('str')
ratings_df['id'].dtype

# Set index as 'id'
ratings_df = ratings_df.set_index('id')
ratings_df.head()

,rating
id,
110,1.0
147,4.5
858,5.0
1221,5.0
1246,5.0


In [29]:
# Groupby id to find mean of user ratings
ratings_group = ratings_df.groupby('id')
# 45115 rows
ratings_mean = ratings_group['rating'].mean().round(2).reset_index()
# Verify data
id_2 = ratings_mean[ratings_mean['id'] == '2']

In [30]:
# Merge ratings and movie adaptations df
final_movies_df = pd.merge(adaptations_merge, ratings_mean, how = 'left', on = 'id')
final_movies_df.head()

,id,keywords,adult,budget,genres,original_language,original_title,release_date,revenue,runtime,spoken_languages,status,title,rating
0,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,NaN
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,Sense and Sensibility,1995-12-13,135000000.0,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Sense and Sensibility,2.72
2,8012,"[{'id': 395, 'name': 'gambling'}, {'id': 416, ...",False,30250000,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",en,Get Shorty,1995-10-20,115101622.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Get Shorty,3.84
3,11859,"[{'id': 258, 'name': 'bomb'}, {'id': 416, 'nam...",False,50000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,Fair Game,1995-11-03,11534477.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Fair Game,NaN
4,39428,"[{'id': 531, 'name': 'southern usa'}, {'id': 8...",False,0,"[{'id': 18, 'name': 'Drama'}]",en,The Neon Bible,1995-08-23,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Neon Bible,NaN


In [10]:
# Drop goodreads_df columns
#goodreads_df = [['']]

In [47]:
# Merge Goodreads data with final_movies_df
goodreads_df = pd.merge(final_movies_df, goodreads_df, how = 'left', on = 'title')
goodreads_df.head()

,id_x,keywords_x,adult_x,budget_x,genres_x,original_language_x,original_title_x,release_date_x,revenue_x,runtime_x,...,authors,average_rating,isbn,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 11
0,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...",False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,1995-12-22,81452156.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,Sense and Sensibility,1995-12-13,135000000.0,136.0,...,Jane Austen/Claudia L. Johnson,4.07,039397751X,eng,269,768.0,68.0,37194,W. W. Norton & Company,NaN
2,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,Sense and Sensibility,1995-12-13,135000000.0,136.0,...,Jane Austen/Claire Lamont/James Kinsley,4.07,192804782,eng,327,6853.0,249.0,38106,Oxford University Press,NaN
3,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,Sense and Sensibility,1995-12-13,135000000.0,136.0,...,Jane Austen/Claire Lamont/Ros Ballaster,4.07,140434259,eng,346,528.0,39.0,35065,Penguin Classics,NaN
4,4584,"[{'id': 420, 'name': 'bowling'}, {'id': 818, '...",False,16500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,Sense and Sensibility,1995-12-13,135000000.0,136.0,...,Jane Austen/Laura Engel,4.07,159308336X,eng,312,1189.0,86.0,38281,Barnes & Noble,NaN
